<a href="https://colab.research.google.com/github/tomonari-masada/course2024-nlp/blob/main/EDA_with_multilingual_e5_large_instruct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tqdm.auto import tqdm
import numpy as np

import spacy
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity

import torch
import torch.nn as nn

from datasets import load_dataset
from transformers import (
    TrainingArguments,
    AutoTokenizer,
    AutoModelForSequenceClassification,
)
from transformers.modeling_outputs import ModelOutput

from trl import SFTTrainer

In [ ]:
dataset = load_dataset(
    "shunk031/livedoor-news-corpus",
    train_ratio=0.8, val_ratio=0.1, test_ratio=0.1,
    random_state=42,
    shuffle=True,
    trust_remote_code=True,
)
num_categories = len(set(dataset["train"]["category"]))
max_seq_length = 512

* https://huggingface.co/intfloat/multilingual-e5-large-instruct

In [ ]:
model_id = "intfloat/multilingual-e5-large-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
pretrained = AutoModelForSequenceClassification.from_pretrained(
    model_id,
    num_labels=num_categories,
).to(0)

/home/masada/anaconda3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at intfloat/multilingual-e5-large-instruct and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
pretrained

XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=1024, out_fe

In [ ]:
pretrained.config

XLMRobertaConfig {
  "_name_or_path": "intfloat/multilingual-e5-large-instruct",
  "architectures": [
    "XLMRobertaModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6,
    "LABEL_7": 7,
    "LABEL_8": 8
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_d

In [ ]:
def accuracy(model, tokenizer, corpus, labels, batch_size=4):
    model.eval()
    num_correct_answers, num_answers = 0, 0
    for i in tqdm(range(0, len(corpus), batch_size)):
        texts = corpus[i:i+batch_size]
        encodings = tokenizer(texts, padding=True, return_tensors="pt")
        encodings = encodings.to(model.device)
        category = torch.tensor(labels[i:i+batch_size]).to(model.device)
        with torch.no_grad():
            outputs = model(**encodings)
        predicted = outputs.logits.argmax(-1)
        num_correct_answers += (predicted == category).sum()
        num_answers += len(texts)
    model.train()
    return (num_correct_answers / num_answers).item()

In [ ]:
def average_pool(last_hidden_states, attention_mask):
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

In [ ]:
def embed(model, tokenizer, corpus, batch_size=4):
    model.eval()
    pooled_hidden_states = []
    for i in tqdm(range(0, len(corpus), batch_size)):
        texts = corpus[i:i+batch_size]
        encodings = tokenizer(texts, padding=True, return_tensors="pt")
        encodings = encodings.to(model.device)
        with torch.no_grad():
            outputs = model.roberta(**encodings)
        pooled_hidden_state = average_pool(
            outputs.last_hidden_state,
            encodings['attention_mask'],
        )
        pooled_hidden_state = pooled_hidden_state.float().cpu().numpy()
        pooled_hidden_states.append(pooled_hidden_state)
    model.train()
    return np.concatenate(pooled_hidden_states)

In [ ]:
embeddings = {}
for key in dataset:
    embeddings[key] = embed(pretrained, tokenizer, dataset[key]["title"])
    embeddings[key] = normalize(embeddings[key])

  0%|          | 0/1474 [00:00<?, ?it/s]

  0%|          | 0/185 [00:00<?, ?it/s]

  0%|          | 0/184 [00:00<?, ?it/s]

In [ ]:
label_pos_tags = ["NOUN", "VERB", "PROPN"]

nlp = spacy.load("ja_core_news_sm")
corpus = {}
for key in dataset:
    corpus[key] = []
    for text in tqdm(dataset[key]["title"]):
        corpus[key].append(" ".join(
            [token.lemma_
             for token in nlp(text) if token.pos_ in label_pos_tags
            ]
        ))

  0%|          | 0/5894 [00:00<?, ?it/s]

  0%|          | 0/737 [00:00<?, ?it/s]

  0%|          | 0/736 [00:00<?, ?it/s]

In [ ]:
corpus["train"][0]

'トレンド マイクロ サービス safesync 無料 体験 横浜 スタジアム 貸切 権 デジカメ 当てる !'

In [ ]:
vectorizer = TfidfVectorizer(min_df=10, max_df=0.1, lowercase=False)
vectorizer.fit(corpus["train"])
vocab = np.array(vectorizer.get_feature_names_out())

In [ ]:
vocab

array(['1人', '1日', '2人', 'AQUOS', 'Android', 'CM', 'CPU', 'DVD', 'Excel',
       'Facebook', 'Fi', 'GHz', 'GX', 'Google', 'HD', 'HT', 'HTC', 'ICS',
       'IS', 'IT', 'In', 'KDDI', 'LAN', 'Mac', 'Mobile', 'NHK', 'NTT',
       'No', 'OK', 'OS', 'Office', 'PC', 'Play', 'REPORT', 'SC', 'SH',
       'SII', 'SNS', 'SO', 'SPORTS', 'SX', 'TV', 'Twitter', 'UP', 'USB',
       'Ubuntu', 'VS', 'Vol', 'Wi', 'Windows', 'Word', 'W杯', 'akb',
       'android', 'arrows', 'au', 'by', 'cafe', 'eluga', 'for', 'galaxy',
       'hulu', 'iOS', 'iPhone', 'iii', 'ipad', 'isw', 'jojo', 'lte',
       'medias', 'mm', 'note', 'nottv', 'optimus', 'pantone', 'peachy',
       'phone', 'presented', 'regza', 'salon', 'siii', 'sim', 'tab',
       'touch', 'ultrabook', 'watch', 'web', 'wimax', 'xi', 'xperia',
       'あらわれる', 'あり', 'ある', 'いう', 'いく', 'いける', 'いる', 'おく', 'お気に入り', 'かける',
       'くさる', 'くす', 'くる', 'くれる', 'こと', 'さん', 'しまう', 'すぎる', 'する', 'たち',
       'ため', 'ちゃん', 'つく', 'つける', 'できる', 'とき', 'とも', 'どっち', 'なし', 'なでし

In [ ]:
vocab_embeddings = embed(pretrained, tokenizer, list(vocab))

  0%|          | 0/213 [00:00<?, ?it/s]

In [ ]:
len(vocab)

850

In [ ]:
n_clusters = 30
kmeans = KMeans(n_clusters=n_clusters, n_init='auto', random_state=123)
kmeans.fit(embeddings["train"])
centers = kmeans.cluster_centers_

In [ ]:
unique, counts = np.unique(kmeans.labels_, return_counts=True)
size_dict = dict(zip(unique, counts))
print(sorted([item[1] for item in size_dict.items()]))

[68, 79, 84, 101, 103, 132, 135, 136, 137, 157, 160, 162, 163, 170, 175, 176, 200, 216, 217, 227, 227, 240, 254, 273, 285, 290, 296, 306, 339, 386]


In [ ]:
topic_words = []
similarities = cosine_similarity(vocab_embeddings, centers)
for i in range(similarities.shape[-1]):
    indices = np.argsort(- similarities[:,i])
    topic_words.append(" ".join(list(vocab[indices[:20]])))
print("\n".join(topic_words))

インテル パソコン シャープ デジ パナソニック 搭載 デュアルコア ソニー モバステ クアッドコア コレ はじめる 機種 トップ サイフ キーボード ニコニコ ホーム 楽天 超える
コレ すぎる しまう 騒動 負ける なる ダメ たち 感ずる ワケ どっち アナタ 込む さん 物議 騒然 ツイッター する なでしこ 超える
アナタ なでしこ 真央 エリカ コレ 優子 オンナ モテる 超える たち 愛す ヒロイン さん くれる みる アナ ドキドキ しまう 真司 イケメン
モテる しまう アナタ 愛す 超える なれる オンナ 込む コレ 彼氏 週間 お気に入り トップ なでしこ くす イケショップ 感ずる ベスト イケメン 今週
イケショップ モバステ ケータイ ドコモ スマホ シャープ 楽天 スマートフォン バッテリー コレ タブレット 携帯 ソニー ソフトバンク 端末 充電 アップル 搭載 モバイル パナソニック
コレ なでしこ くれる イケショップ アナタ バレンタイン らくらく ホーム お気に入り 愛す しまう にゅう ベルセルク 癒す よめ エリカ 超える 過ごす ゲット トップ
コレ アナタ しまう 超える なる たち くれる なでしこ 込む おく する よる なれる どっち さん みる ウラ くす 虎の巻 感ずる
コレ 選手 負ける なでしこ ワケ 圭佑 観る どっち アナタ 超える ツイッター なる しまう 狙う たち プレーヤー みる W杯 見る よる
虎の巻 裏技 はじめる まとめる トップ 使い方 コレ ワザ しまう フラッシュバック まとめ 入れる サイフ 込む 付ける 隠す する ツイッター 使う ニコニコ
虎の巻 ハリウッド 上映 アベンジャーズ ムービー アナタ 映画 コレ 超える 観る 必見 ディズニー フジテレビ フラッシュバック 登場 復活 ワンセグ 主演 新作 なでしこ
ニコニコ ツイッター ツイート コレ ネット はじめる トップ しまう サイト する ホーム アナタ 楽天 ブログ インターネット サイフ どっち みる 超える モバステ
コレ アナタ なでしこ しまう 負ける 泣く 真央 ワケ たち 超える なる すぎる エリカ 虎の巻 さん オンナ 真司 込む フジテレビ 感ずる
どっち アナタ モテる しまう オンナ コレ

In [ ]:
X_train = vectorizer.transform(corpus["train"]).toarray()
vocab_embeddings = np.dot((X_train / X_train.sum(0)).T, embeddings["train"])

In [ ]:
topic_words = []
similarities = cosine_similarity(vocab_embeddings, centers)
for i in range(similarities.shape[-1]):
    indices = np.argsort(- similarities[:,i])
    topic_words.append(" ".join(list(vocab[indices[:20]])))
print("\n".join(topic_words))

PC デジ パソコン 登場 新型 ノート 製品 シリーズ 使える 機能 HD ultrabook 液晶 Windows 使う ディスプレイ データ 発売 対応 販売
批判 殺到 非難 物議 続出 次ぐ 発言 ネット 賛否 騒動 怒り 掲示 激怒 コメント 波紋 問題 受ける 疑問 炎上 ユーザー
さん 披露 美女 ある いく いる こと インタビュー くる 見る ちゃん 美人 出演 女優 注目 すぎる 少女 人気 する デビュー
週間 ランキング ライフスタイル 登録 お気に入り 記事 ビューティー 恋愛 行動 方法 ビューティ 彼氏 効く 作る なれる みんな つける 女子 なる する
ケース イケショップ レア 専用 カバー スマホ バッテリー チェック 使える for iPhone 充電 グッズ アイテム 容量 機能 使う 紹介 登場 売れ筋
気分 楽しむ プレゼント 入れる 行く 本格 人気 癒す ため 大人 なる くれる 贈る 期間 誕生 今年 特集 注目 作る 見る
ある いう する こと いる なる つく 出る 思う いく やる もの 見る 取る さん くる 時代 くれる 変わる 出す
watch SPORTS 選手 試合 監督 プロ 代表 星野 チーム 野球 巨人 語る なでしこ 本田 サッカー 開幕 岡田 ノム スポーツ アナ
虎の巻 得る ワザ 知る Word ファイル Excel テクニック 便利 入力 管理 表示 役立つ 操作 活用 裏技 使う 方法 使い方 Facebook
映画 作品 主演 上映 公開 国際 ストーリー プレミア 描く 映像 アカデミー 解禁 まとめ 感動 受賞 決定 ハリウッド 編集 超える 予告
情報 話題 利用 サービス 使う わかる みる サイト なる よる できる つく メディア web あり ユーザー ニュース 公式 使える する
さん 発言 対する 騒然 暴露 出演 ファン コメント いく 出る いる 騒動 告白 明かす 続出 受ける akb 関係 次ぐ 報道
男性 いる もの 事情 する 本当 なる しまう 思う ある こと アリ 女性 とき 聞く 結婚 いう 自分 求める たち
開始 実施 サービス 通信 キャンペーン 端末 利用 向け モバイル ドコモ 購入 回線 対応 料金 スマホ ソフトバンク NT

In [ ]:
class MyNetForClassification(nn.Module):
    def __init__(self, pretrained):
        super().__init__()
        self.pretrained = pretrained
        self.config = self.pretrained.config

    def forward(
        self, input_ids, category=None,
        attention_mask=None,
        output_attentions=None, output_hidden_states=None,
        return_dict=None, inputs_embeds=None, labels=None,
    ):
        outputs = self.pretrained(
            input_ids,
            attention_mask=attention_mask,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        loss_fct = nn.CrossEntropyLoss()
        loss = loss_fct(outputs.logits, category)
        return ModelOutput(
            loss=loss,
            logits=outputs.logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )
model = MyNetForClassification(pretrained)

In [ ]:
training_args = TrainingArguments(
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    output_dir="outputs",
    label_names=["category"],
    max_steps=500,
    eval_steps=100,
    logging_steps=100,
    save_steps=100,
    learning_rate=5e-5,
    optim_target_modules=["query", "key", "value", "dense"],
    evaluation_strategy="steps",
    logging_strategy="steps",
    save_strategy="steps",
    load_best_model_at_end=True,
)

In [ ]:
trainer = SFTTrainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    max_seq_length=max_seq_length,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    dataset_text_field="title",
)
trainer.train_dataset = trainer.train_dataset.add_column(
    "category", dataset["train"]["category"],
)
trainer.eval_dataset = trainer.eval_dataset.add_column(
    "category", dataset["validation"]["category"],
)

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
trainer.train()

Step,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
batch_dict = tokenizer(dataset["validation"]["title"][:10], max_length=512, padding=True, truncation=True, return_tensors='pt')

In [ ]:
outputs = pretrained(**batch_dict.to(0))

In [ ]:
outputs.logits.argmax(-1)

tensor([2, 0, 4, 2, 1, 0, 7, 2, 6, 1], device='cuda:0')

In [ ]:
dataset["validation"]["category"][:10]

[2, 0, 4, 2, 1, 0, 7, 2, 6, 2]

In [ ]:
accuracy(pretrained, tokenizer, dataset["validation"]["title"], dataset["validation"]["category"])

  0%|          | 0/185 [00:00<?, ?it/s]

0.9158751964569092

In [ ]:
embeddings = {}
for key in dataset:
    embeddings[key] = embed(pretrained, tokenizer, dataset[key]["title"])
    embeddings[key] = normalize(embeddings[key])

  0%|          | 0/1474 [00:00<?, ?it/s]

  0%|          | 0/185 [00:00<?, ?it/s]

  0%|          | 0/184 [00:00<?, ?it/s]

In [ ]:
label_pos_tags = ["NOUN", "VERB", "PROPN"]

nlp = spacy.load("ja_core_news_sm")
corpus = {}
for key in dataset:
    corpus[key] = []
    for text in tqdm(dataset[key]["title"]):
        corpus[key].append(" ".join(
            [token.lemma_
             for token in nlp(text) if token.pos_ in label_pos_tags
            ]
        ))

  0%|          | 0/5894 [00:00<?, ?it/s]

  0%|          | 0/737 [00:00<?, ?it/s]

  0%|          | 0/736 [00:00<?, ?it/s]

In [ ]:
vectorizer = TfidfVectorizer(min_df=10, max_df=0.1, lowercase=False)
vectorizer.fit(corpus["train"])
vocab = np.array(vectorizer.get_feature_names_out())
X = {}
for key in dataset:
    X[key] = vectorizer.transform(corpus[key]).toarray()
vocab_embeddings = np.dot((X["train"] / X["train"].sum(0)).T, embeddings["train"])

In [ ]:
n_clusters = 30
kmeans = KMeans(n_clusters=n_clusters, n_init='auto', random_state=123)
kmeans.fit(embeddings["train"])
centers = kmeans.cluster_centers_

In [ ]:
unique, counts = np.unique(kmeans.labels_, return_counts=True)
size_dict = dict(zip(unique, counts))
print(sorted([item[1] for item in size_dict.items()]))

[57, 59, 79, 91, 95, 98, 100, 102, 111, 115, 125, 126, 146, 150, 151, 168, 169, 175, 218, 220, 225, 239, 259, 264, 285, 301, 333, 388, 491, 554]


In [ ]:
topic_words = []
similarities = cosine_similarity(vocab_embeddings, centers)
for i in range(similarities.shape[-1]):
    indices = np.argsort(- similarities[:,i])
    topic_words.append(" ".join(list(vocab[indices[:20]])))
print("\n".join(topic_words))
#with open("topic_words.txt", "w") as f:
#    f.write("\n".join(topic_words))

PC ultrabook ファイル ソフト 虎の巻 Mac 得る フラッシュバック マザー ワザ ニコニコ Excel 募集 レビューアー Word Ubuntu ノート ロゴ 会議 デジ
上陸 プレミア DVD ナイト ダーク 解禁 主演 ポスター 映像 ストーリー スター ハリウッド アカデミー 主題 描く 受賞 予告 ベルセルク 映画 来日
続出 非難 物議 次ぐ 騒然 中島 殺到 賛否 両論 発言 橋下 批判 掲示 母親 反応 怒り 波紋 暴露 疑問 騒動
イー インチ スペック lte らくらく スマートフォン 向け wimax galaxy Android Wi Fi siii ドコモ SX 開始 GX AQUOS プラチナ phone
伝統 大人 学ぶ アイテム 過ごす 気分 デート プロジェクト ファッション 入れる 愛す 選ぶ 食べる 効く ムービー ライフ なれる パワー 楽しむ 秘訣
watch SPORTS 星野 斎藤 ダルビッシュ 巨人 ノム 開幕 本田 長友 試合 W杯 圭佑 岡田 移籍 落合 代表 なでしこ 選手 チーム
売れ筋 アップル 家電 話題 パナソニック チェック インターネット 売れる 発生 今度 電気 ソニー hulu 原因 節電 電池 デジカメ salon プレーヤー 未来
独女 オトナ コミック 悩み ゆるい presented cafe オンナ by よめ あらわれる くさる ひとり はじめる たち しまう 風呂 アリ 結婚 女子
ビデオ hulu salon パナソニック 連載 家電 連動 節電 デジカメ 電気 ソニー インターネット 未来 売れ筋 USB プレーヤー 売れる メーカー アップル チェック
ビジネスマン 年収 説教 辛口 部屋 図鑑 ウラ 株式 人事 スーツ プレイヤー 活動 転職 ゴルフ ビジネス オトコ 研究 刺激 解決 コラム
ポスター アベンジャーズ 予告 ハリウッド ベルセルク 来日 ダーク 人類 解禁 映像 ナイト 公開 劇場 エンター 主題 スター 読み 対決 週末 批評
プレゼント 終了 バレンタイン スイーツ メイク peachy ケーキ 登録 コスメ 気分 美肌 ビューティ 週間 ビューティー ランキング セット お気に入り UP トレンド ガール
ガール 招待 ケーキ バレンタ